
<span type="title">多态</span> | <span type="update">2018-07-20</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要讲解多态。</p></span>

面向对象的程序设计语言中，三大基本特征是 `数据抽象、继承和多态`。

其中对于抽象而言，就是将对象的特征和行为封装起来。“封装” 通过这种归类来创造新的数据类型 `xxx.class`，其构造了新的数据类型。“实现隐藏” 通过权限关键字实现了对于接口和实现（细节）的分离。而继承，则提供了一种代码重用的方法，而在代码重用的时候，最容易出现的问题就是类型之间的耦合（父类、子类和它类），因此，“多态” 就是解决这种耦合的的重要工具。

这是一种重要的能力，它允许将由同一个类继承而来的多个类型当作同一类型来处理，因此，只用写一份代码，就可以在由这个类型导出的任何类型上使用。从某种程度上，继承和多态的关系，是“变与不变”的关系。

因此，我觉得对于Java的OO，三大基本特征应该是 `数据抽象、实现隐藏、继承多态`

# 向上转型

向上转型来自于UML的箭头，本来是从父类指向子类的箭头反向上指，因此称作向上转型。向上转型允许我们定义一个基类的类型，然后使用子类的构造器创建一个对象，这个对象将“向上转型”为基类使用。

```java
//show how to use up casting
import static com.mazhangjing.Print.*;
public class Game {
	private Cycle cycle;
	public Game(Cycle c){
		this.cycle = c;
	}
	public void ride(){
		print("In Game.rideBy()");
		cycle.ride();
	}
	public static void main(String[] args){
		Game a = new Game(new Unicycle());
		a.ride();//up casting
	}
}
class Cycle {
	public void ride() {
		print("Cycle ride");
		wheels();
	}
	public void wheels() {
		print("The cycle have 4 wheels");
	}
}
class Unicycle extends Cycle {
	public void ride() {
		print("Unicycle ride");
	}
}
class Bicycle extends Cycle {
	public void ride() {
		print("Bicycle ride");
	}
}
class Tricycle extends Cycle {
	public void ride() {
		print("Tricycle ride");
	}
}

In Game.rideBy()
Unicycle ride
```


如上所示，在 Game 的 ride 方法中，其接受的是 Cycle 类，而现在处理的是 Unicycle 类，这就发生了向上转型。向上转型里发生的事情很有意思，当基类和子类共有一个方法被调用，ride显示了其结果——子类的方法被返回，因此这种机制可以让我们只关注基类，而不用担心导出类，当导出类重载其方法时，返回导出类方法，反之，则返回基类方法。

# 动态绑定

当进行向上转型的时候，其实发生了动态绑定。

在C语言中，所有的代码在被编译的时候就已经绑定了，其称之为前向绑定，因此方法体.方法名是唯一的，指向唯一一段代码，在程序运行期间不可更改。但是Java的绑定（方法体和方法名）是动态，或者说是运行时绑定，编译器在编译的时候一直不知道对象的类型，但是在之后，方法调用机制会判断正确对象的类型，并且调用恰当的方法。

Java中除了 static 和 final(private属于final)之外，都是动态绑定的。在早期，可能将一个方法声称为 final 为了获得更好的性能，关闭动态绑定，但是，这对于程序整体性能影响不大。这一点需要注意。

对于上述例子：

`Cycle x = new Unicycle()` 就是一个动态绑定的例子，在这个声明中，当 `Unicycle()` 返回一个对象的时候，因为通过继承 Unicycle 本身就是一种 Cycle，因此这句话没有错误。这是一个向上转型。

好玩的事情发生在这句话里：

`x.ride()` 现在，调用了 `ride()` 方法，因为在上一句我们知道 x 一定是 Cycle的一个引用（根据语法定义），那么这个 `ride` 必须在 `Cycle` 基类被实现，但是真正调用的时候，也就是这个 `.`符号 起作用的时候，Java会寻找合适的一个对象，结果它找到了`Unicycle` 这个对象的一个实例 `x`，因此，就使用这个实例来进行 `ride` 的方法调用。

在这里就发生了向上转型和动态绑定。

需要注意，对于这个方法，一定在基类中被实现，对于这个方法的真实调用，如果子类进行了重载，那么会因为动态绑定使用子类的对象实例，因此使用了子类的重载过的方法。

```java
//use to show dynamic binding
import static com.mazhangjing.Print.*;
import java.util.*;
public class Shapes {
	public static void main(String[] args){
		RandomShape rs = new RandomShape();
		Shape[] s = new Shape[5];
		for (int i = 0 ; i < s.length; i++){
			s[i] = rs.next();
		}
		for (Shape i : s){
			i.draw();
			i.author();
		}
		
		Shape c = new Cycle(4);
		//print(c.wheels());//error, c is a Shape quote, the compiler will
		//first find wheels() in Shape class, and because of dynamic binding
		//it will call Cycle().xxx method, the compiler don't know it.
		c.draw(); // right
	}
}
class Shape {
	public void draw() {print("Shape is drawing...");}
	public void author() {print("By Corkine Ma");}
}
class Line extends Shape {
	@Override public void draw() {print("Line is drawing...");}
	public void author() {print("By Corkine Ma in Line");}
}
class Cycle extends Shape {
	private int wheels_num;
	Cycle(int num){
		wheels_num = num;
	}
	@Override public void draw() {print("Cycle is drawing...");}
	public int wheels(){
		return wheels_num;
	}
}
class Triangle extends Shape {
	public void draw() {print("Triangle is drawing...");}
}
class RandomShape {
	private Random rand = new Random();
	public Shape next(){
		switch (rand.nextInt(3)){
			default:
			case 0: return new Line();
			case 1: return new Cycle(rand.nextInt(3));
			case 2: return new Triangle();
		}
	}
}
```

如上面例子所示，RandomShape的next()方法在Shapes调用时返回的是Shape类型，在真实调用时，不论是Line还是Cycle还是Triangle，都发生了向上转型。

而在draw方法发生的时候，则进行了动态绑定，调用子类实例方法体，返回子类中重载的方法。

需要注意注释的地方，如果一个方法在子类中有，但在父类中没有，那么不可调用，这很简单，因为这里的c是一个Shape，编译器无法在Shape中找到这个wheels()的方法。

因此，现在我们知道了变与不变，其中的变指的是动态绑定，给不同的方法体，可以调用其内部不同的重载，不变指的是向上转型，调用必须在基类中实现，如wheels()方法一样，因为只有这样，我们在使用wheels()方法时，不用担心继承类是如何处理的，因为不论怎样，如果找不到重载，总是会返回基类的这个方法。

## 可扩展性

如下所示是一个例子，其中Instrument表示乐器基类，其play方法接受一个Note曲谱。这个基类有一个adjust()方法来调弦。现在我们通过继承构造了几种不同的乐器，比如Wind，Percussion等，对于Wind也进行了继承，这杯用来表示复杂的“变”的结构。在这些子孙类中，有一些重载了adjust()和play()方法，有一些没有，但不论怎样，在最后的Music类中，对Instrument基类调用play()方法总不会出错，因为如果在子类没有实现，那么回转回头调用基类的定义，因为我们已经要求过这种动态绑定必须在基类中实现，因此程序具有了丰富的可扩展性，并且不会出错。

注意 `print(orchestra[4]);` 这句话，在这个类中是没有`toString`实现的，但没有发生任何错误。

```java
//use to show dynamic binding advanced tech
import static com.mazhangjing.Print.*;
import java.util.*;
enum Note {
	MIDDLE_C, C_SHAPE, B_FLAT;
}
class Instrument {
	void play(Note n) {print("Playing Instrument "+n);}
	public String toString() {return "Instrument";}
	void adjust() {print("Adjusting Instrument");}
}
class Wind extends Instrument {
	void play(Note n) {print("Wind.play() " + n);}
	String what() {return "Wind";}
	void adjust() {print("Adjusting Wind");}
}
class Percussion extends Instrument {
	void play(Note n) {print("Percussion.play() " + n);}
	String what() {return "Percussion";}
	void adjust() {print("Adjusting Percussion");}
}
class Stringed extends Instrument {
	void play(Note n) {print("Stringed.play() " + n);}
	String what() {return "Stringed";}
	void adjust() {print("Adjusting Stringed");}
}
class Brass extends Wind {
	//void play(Note n) {print("Brass.play() "+n);}
	void adjust() {print("Adjusting Brass");}
}
class Woodwind extends Wind {
	void play(Note n) {print("Woodwind.play() "+n);}
	void adjust() {print("Adjusting Woodwind");}
}
public class Music {
	public static void tune(Instrument i){
		i.play(Note.MIDDLE_C);
	}
	public static void turnAll(Instrument[] e){
		for (Instrument i : e){
			tune(i);
		}
	}
	public static void main(String[] args){
		Instrument[] orchestra = {
			new Wind(),
			new Percussion(),
			new Stringed(),
			new Brass(),
			new Woodwind()
		};
		turnAll(orchestra);
		print(orchestra[4]);
		
		RandomIns x = new RandomIns();
		x.next().adjust();
	}
}
class RandomIns {
	private Random rand = new Random();
	public Instrument next() {
		switch (rand.nextInt(5)){
			default:
			case 0: return new Wind();
			case 1: return new Percussion();
			case 2: return new Stringed();
			case 3: return new Brass();
			case 4: return new Woodwind();
		}
	}
}

Wind.play() MIDDLE_C
Percussion.play() MIDDLE_C
Stringed.play() MIDDLE_C
Wind.play() MIDDLE_C
Woodwind.play() MIDDLE_C
Instrument
Adjusting Wind
```

## 缺陷：覆盖私有方法

下列代码展示了一个很有意思的问题，当发生向上转型和动态绑定的时候，也就是当 `Father x = new Fault()` 调用的时候，x 调用 f() 方法将不会发生正确的动态绑定，因为 private 有 final 的意味。这里好玩的是，如果你对于子类调用父类的方法，如果子类没有重载，那么会直接调用父类，就像这里发生的一样。但是，这里声明的是 private，也就是说，如果真的子类没有，那么它是不允许使用父类的 private 方法的，因为父类对其屏蔽这个方法。

但是按照总的来说，将其理解为：因为 final 限定了不能继承，因此子类的相同名称方法不能看作重载，所以直接返回了父类的方法。

可能对于动态绑定，priavte 被某种程度上看做了 final。

因此，这里的教训就是，只有非private方法才可以被覆盖。或者为了清晰，直接在子类添加一个@Override去重载，如果出错，就知道哪里错了。

```java
//private method fault in up casting
import static com.mazhangjing.Print.*;
class Fault extends Father {
    //@Override 
	public void f() {
		print("Public f() in Falut");
	}
	public void g() {
		print("Public g() in Falut");
	}
}
public class Father {
	private void f(){
		print("Private f() in father");
	}
	public void g(){
		print("Public g() in father");
	}
	public static void main(String[] args){
		Father x = new Fault();
		x.f(); //Private f() in father
		x.g(); //Public g() in Falut
	}
}
Private f() in father
Public g() in Falut
```

## 缺陷：域和静态方法

动态绑定不能去重载子类中的域和静态方法，从某种程度上来说，这算是一种缺陷。

```java
//show the static method and the field 's relationship with dynamic binding
import static com.mazhangjing.Print.*;
public class People extends Super {
	int i = 30;
	static void getInfo(){
		print("People.static getInfo method");
	}
	public void getInfo2(){
		print("People.public getInfo method");
	}
	public static void main(String[] args){
		Super x = new People();
		print(x.i);
		x.getInfo();
		x.getInfo2();
	}
}
class Super {
	int i = 20;
	static void getInfo() {
		print("Super.static getInfo method");
	}
	public void getInfo2(){
		print("Super.public getInfo method");
	}
}
20
Super.static getInfo method
People.public getInfo method
```

如上的代码所示，返回的是父类的域和父类的静态方法，而对于动态方法，则正确进行了绑定。

对于私有方法的覆盖、静态方法和域的无效的多态机制是Java固有的缺陷。因为按照绝对正确的模型，这样的写法会调用子类相关的域和静态方法，但是出于某种原因，当给方法名安上方法体时，Java只对那些动态方法进行了多态和动态绑定，而对于其余的——私有、静态方法和域，都默认按照基类进行处理。

## Java多态机制深究

下面这个例子很好的说明了Java多态是如何进行的/机制问题（对于动态方法而言）。

```java
//use to show up casting advanced tech II
import static com.mazhangjing.Print.*;
class Father {
	void methodA(){
		print("Father-methodA");
	}
	void methodB(){
		this.methodA();
		print("Father-methodB");
	}
}
public class Son extends Father{
	@Override void methodA() {
		print("Son-methodA");
	}
	void findFather(Father f){
		f.methodB();
	}
	public static void main(String[] args){
		Son s = new Son();
		s.findFather(s);
	}
}
Son-methodA
Father-methodB
```

这个例子展示了，当发生方法重载的时候，如果向上提升之后的对象调用一个没有被重载的方法——会返回基类的对应方法。但是，如果这个基类的对应方法链接了另一个基类方法，而这个基类方法在子类中被重载了呢？

这里可以让我们更清晰的看出Java的多态机制模型。这个子类中暗示了其含有父类所有的代码，然后对某些代码进行了重载，当发生动态绑定的时候，s被安上了一个叫做 Son 的头，而这个 Son 中存在的（重载的）动态方法被调用，那些“不存在”的方法，则由父类提供（这种看法是错误的，准确的表述应该是那些你在子类中由于眼瞎看不见但隐式存在的由父类继承的没有被子类重载的方法被调用），因此，这里的 `this` 其实是在子类的实例中被执行，其代指的是子类，所以嗲用 methodA 的时候，返回的是子类的方法。